In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyspark
from datetime import datetime, timedelta
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.window import Window
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.types import FloatType
from pyspark.sql.functions import when, col, to_date, lag
from pyspark.sql.functions import to_timestamp, count, isnan, isnull, mean, rand
from pymongo import MongoClient

# Models
from pyspark.ml.regression import LinearRegression
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet, plot


%matplotlib inline
plt.style.use('fivethirtyeight')

In [14]:
# Load Data for models
mongo_uri = "mongodb://hadoop-vm.internal.cloudapp.net:27017/ca2"
client = MongoClient(mongo_uri)
db = client["ca2"]
collection = db["vaccin_tweets_analysis"]

# Convert MongoDB collection to pandas DataFrame
df = pd.DataFrame(list(collection.find()))
df = df[["timestamp","sentiment_score","is_retweet"]]
df["timestamp"] = df["timestamp"].apply(lambda x: pd.to_datetime(x).date())
df.rename(columns={"timestamp": "date"},inplace=True)

# Aggregate daily data
data = df.groupby('date').agg(avg_sentiment_score=('sentiment_score', 'mean'))
data['retweet_rate'] = df.groupby('date')['is_retweet'].mean()
data.reset_index(inplace=True)

In [15]:
start_date = pd.to_datetime("2020-04-01").date()
split_date = pd.to_datetime("2021-01-01").date()
end_date = pd.to_datetime("2021-05-01").date()

train = data.loc[(data.date >= start_date) & (data.date <= split_date)]
test = data.loc[(data.date >= split_date) & (data.date <= end_date)]

In [16]:
train.head()

,date,avg_sentiment_score,retweet_rate
0,2020-04-01,0.505337,0.632184
1,2020-04-02,0.495604,0.632787
2,2020-04-03,0.514191,0.641686
3,2020-04-04,0.249112,0.724858
4,2020-04-05,0.260608,0.696226


## Linear Regression Model

In [17]:
def build_vector(lag_periods, df_grouped, skip_to):
    window = Window.orderBy("date")
    
    # Create groups for 1,2,3,4,5,6,7 weekly seasons.
    for i in lag_periods:
        df_grouped = df_grouped.withColumn(f'sentiment_score_lag_{i}', lag(col('avg_sentiment_score'), i).over(window))
        
    # Filter out the first 7 rows with NAN
    filter_d = f"date > date'{skip_to}'"
    df_grouped = df_grouped.filter(filter_d)
    
    input_cols = [f'sentiment_score_lag_{i}' for i in lag_periods]
    input_cols.append("retweet_rate")
    input_cols
    
    # Assembler Vector - A feature transformer that merges multiple columns into a vector column.
    # https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.ml.feature.VectorAssembler.html
    va = VectorAssembler(
        inputCols=input_cols,
        outputCol='features')
    
    return va.transform(df_grouped)

In [18]:
def get_linear_regresion_model(data):
    spark = SparkSession.builder.appName('Tweets').getOrCreate()
    spark_df = spark.createDataFrame(data)
    va_df = build_vector([1, 14, 28, 42, 56, 70, 84], spark_df, (start_date + timedelta(days=84+3)))
    train = va_df.filter(va_df.date <= split_date)
    test = va_df.filter(va_df.date > split_date)
    
    # Linear regression model
    lr = LinearRegression(featuresCol='features', labelCol='avg_sentiment_score')

    # Fit the model
    model = lr.fit(train)

    # Make predictions
    predictions = model.transform(test)

    # Evaluate model
    evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='avg_sentiment_score')

    print("MSE: ", model.summary.meanSquaredError)
    print("MAE: ", model.summary.meanAbsoluteError)
    print("R-squared: ", model.summary.r2) 
    print(f"RMSE: {evaluator.evaluate(predictions)}")
    
    return 

In [19]:
spark = SparkSession.builder.appName('Tweets').getOrCreate()
spark_df = spark.createDataFrame(data)
spark_df.printSchema()

root
 |-- date: date (nullable = true)
 |-- avg_sentiment_score: double (nullable = true)
 |-- retweet_rate: double (nullable = true)



In [20]:
spark_df.select("date").show()

Py4JJavaError: An error occurred while calling o108.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2) (host.docker.internal executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:192)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:166)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:364)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:328)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadChe

In [ ]:
lr_model = get_linear_regresion_model(train)